In [1]:
import numpy as np
import pandas as pd

### Import Dataset

In [2]:
## Construct a dataframe from the csv dataset
colnames=['year', 'month', 'day', 'RMM1', 'RMM2','phase','amplitude','Final_value']
df = pd.read_csv('../data/mjo_recent_data.csv', sep=r'[ \t]+', names=colnames, skiprows=[0])
total_days = len(df)

## Convert the columns of the dataframe to the numpy array
year = df['year'].to_numpy()
month = df['month'].to_numpy()
day = df['day'].to_numpy()

RMM1 = df['RMM1'].to_numpy(dtype='float64')
RMM2 = df['RMM2'].to_numpy(dtype='float64')
phase = df['phase'].to_numpy(dtype='float64')
amplitude = df['amplitude'].to_numpy(dtype='float64')

print(df)

       year  month  day      RMM1      RMM2  phase  amplitude  \
0      1974      6    1  1.634470  1.203040      5   2.029480   
1      1974      6    2  1.602890  1.015120      5   1.897290   
2      1974      6    3  1.516250  1.085510      5   1.864760   
3      1974      6    4  1.509810  1.035730      5   1.830920   
4      1974      6    5  1.559060  1.305180      5   2.033260   
...     ...    ...  ...       ...       ...    ...        ...   
17741  2022     12   27  1.520007  0.870776      5   1.751763   
17742  2022     12   28  1.375905  0.978738      5   1.688503   
17743  2022     12   29  0.944452  1.353248      6   1.650233   
17744  2022     12   30  0.539961  1.550516      6   1.641846   
17745  2022     12   31  0.279515  1.511038      6   1.536673   

                           Final_value  
0       Final_value:__OLR_&_NCEP_winds  
1       Final_value:__OLR_&_NCEP_winds  
2       Final_value:__OLR_&_NCEP_winds  
3       Final_value:__OLR_&_NCEP_winds  
4       Final_

/var/folders/51/_9h2_qzn4b54cgs0qgwxr0vh0000gn/T/ipykernel_67238/2965940127.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('../data/mjo_recent_data.csv', sep=r'[ \t]+', names=colnames, skiprows=[0])


### Delete missing values and their preceding values

In [3]:
raw_datas = [RMM1,RMM2,phase,amplitude]
data_names = ['RMM1', 'RMM2', 'phase', 'amplitude']
idx_miss = np.where(RMM1 > 1e34)[0] # index of missing value
n_miss = len(idx_miss) # the number of missing value

## Delete the missing values and their predceding values
new_datas = [0]*4
for i in range(4):
    raw_data = raw_datas[i]
    data_name = data_names[i]
    dat = np.delete(raw_data, np.arange(max(idx_miss)+1)) # np.delete(raw_datas[i], idx_miss)
    new_datas[i] = dat
    if i == 0:
        RMM1_new = dat
    if i == 1:
        RMM2_new = dat
    if i == 2:
        phase_new = dat
    if i == 3:
        amplitude_new = dat

raw_ymds = [year, month, day]
ymd_names = ['year', 'month', 'day']
new_ymds = [0]*3
for i in range(3):
    raw_ymd = raw_ymds[i]
    tmd_name = ymd_names[i]
    dat_ymd = np.delete(raw_ymd, np.arange(max(idx_miss)+1)) # np.delete(raw_ymds[i], idx_miss)
    new_ymds[i] = dat_ymd
    if i == 0:
        year_new = dat_ymd
    if i == 1:
        month_new = dat_ymd
    if i == 2:
        day_new = dat_ymd
    

print(new_datas)
print(RMM1.shape)
print(max(idx_miss)+1)
print(RMM1_new.shape)

print(new_ymds)
print(year.shape)
print(year_new.shape)

[array([ 0.142507  , -0.2042    , -0.15861   , ...,  0.94445157,
        0.53996062,  0.27951464]), array([1.05047  , 1.37361  , 1.53943  , ..., 1.353248 , 1.5505164,
       1.5110376]), array([6., 7., 7., ..., 6., 6., 6.]), array([1.0600899, 1.3887   , 1.54758  , ..., 1.650233 , 1.6418461,
       1.5366727])]
(17746,)
1675
(16071,)
[array([1979, 1979, 1979, ..., 2022, 2022, 2022]), array([ 1,  1,  1, ..., 12, 12, 12]), array([ 1,  2,  3, ..., 29, 30, 31])]
(17746,)
(16071,)


### Save and load the updated dataset

In [4]:
kwds = {'year':year_new, 'month':month_new, 'day':day_new, 'RMM1':RMM1_new, 'RMM2':RMM2_new,'phase':phase_new,'amplitude':amplitude_new}
np.savez('../data/mjo_new_data.npz', **kwds)

npzfile = np.load('../data/mjo_new_data.npz', allow_pickle=True)
print(npzfile.files)
print(npzfile['RMM1'])
print(npzfile['RMM2'])
print(npzfile['year'])
print(npzfile['month'])
print(npzfile['day'])

['year', 'month', 'day', 'RMM1', 'RMM2', 'phase', 'amplitude']
[ 0.142507   -0.2042     -0.15861    ...  0.94445157  0.53996062
  0.27951464]
[1.05047   1.37361   1.53943   ... 1.353248  1.5505164 1.5110376]
[1979 1979 1979 ... 2022 2022 2022]
[ 1  1  1 ... 12 12 12]
[ 1  2  3 ... 29 30 31]
